In [4]:
import math
import logging
from typing import Union
from urllib.parse import urlparse, parse_qsl, urlsplit

import pandas as pandas
import seaborn as sb
import json
import requests
import fhirclient

from IPython.display import display, HTML


In [5]:
#Constants
HAPI_URL = 'http://hapi.fhir.org/baseR4'

In [ ]:
#Configure settings
logging.basicConfig(filename="newfile.log", 
                    format='%(asctime)s %(message)s', 
                    filemode='w')
logger=logging.getLogger()
logger.setLevel(logging.DEBUG)

In [7]:
#Test library on HAPI-FHIR test server
resp = requests.get(HAPI_URL)
print(resp.json())

{'resourceType': 'OperationOutcome', 'text': {'status': 'generated', 'div': '<div xmlns="http://www.w3.org/1999/xhtml"><h1>Operation Outcome</h1><table border="0"><tr><td style="font-weight: bold;">ERROR</td><td>[]</td><td><pre>This is the base URL of FHIR server. Unable to handle this request, as it does not contain a resource type or operation name.</pre></td>\n\t\t\t\t\t\n\t\t\t\t\n\t\t\t</tr>\n\t\t</table>\n\t</div>'}, 'issue': [{'severity': 'error', 'code': 'processing', 'diagnostics': 'This is the base URL of FHIR server. Unable to handle this request, as it does not contain a resource type or operation name.'}]}
